In [11]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.guid
import ifcopenshell.util
import ifcopenshell.util.element
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json
from rdflib import Graph, Literal, Namespace, RDF, RDFS, OWL
import uuid
from pathlib import Path
from itertools import combinations

In [12]:
# Configuration of Namespaces for RDF graph
BRICK = Namespace("https://brickschema.org/schema/Brick#")
BOT = Namespace("https://w3id.org/bot#")
INST = Namespace("https://lbd.example.com/")
PROPS = Namespace("http://lbd.arch.rwth-aachen.de/props#") 

PREFIXES = {
    "brick": BRICK, "bot": BOT, "inst": INST, "rdfs": RDFS, "owl": OWL, "props": PROPS
    }

In [33]:
# --------------------------------------------------
# 1️⃣ Geometry Settings
# --------------------------------------------------

def create_geometry_settings():
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    return settings


# --------------------------------------------------
# 2️⃣ Get Spaces Grouped by Storey
# --------------------------------------------------

def get_spaces_by_storey(ifc):
    """
    Fetches Spaces grouped by Building Storey.
    Uses 'IsDecomposedBy' -> 'IfcRelAggregates'
    """
    storey_spaces = {}
    prefixes_lower = ("a", "b", "c", "d")

    for storey in ifc.by_type("IfcBuildingStorey"):
        spaces = []
        
        # Spaces are AGGREGATED into storeys
        if hasattr(storey, "IsDecomposedBy"):
            for rel in storey.IsDecomposedBy:
                if rel.is_a("IfcRelAggregates"):
                    for obj in rel.RelatedObjects:
                        if obj.is_a("IfcSpace"):
                            # Apply the specific naming and exclusion rules
                            if obj.Name and obj.Name[0].lower() in prefixes_lower:
                                if obj.GlobalId != "2eKm1MRbj9lQ$fk6CjSQZn":  
                                    spaces.append(obj)
                                    
        storey_spaces[storey.GlobalId] = spaces

    return storey_spaces

def get_walls_by_storey(ifc):
    """
    Fetches Walls grouped by Building Storey.
    Uses 'ContainsElements' -> 'IfcRelContainedInSpatialStructure'
    Handles both IfcWall and IfcWallStandardCase.
    """
    storey_walls = {}

    for storey in ifc.by_type("IfcBuildingStorey"):
        walls = []
        
        # Walls are CONTAINED in storeys
        if hasattr(storey, "ContainsElements"):
            for rel in storey.ContainsElements:
                if rel.is_a("IfcRelContainedInSpatialStructure"):
                    # Note: Contained elements are under 'RelatedElements', not 'RelatedObjects'
                    for obj in rel.RelatedElements:
                        if obj.is_a("IfcWall") or obj.is_a("IfcWallStandardCase"):
                            walls.append(obj)
                            
        storey_walls[storey.GlobalId] = walls

    return storey_walls


# --------------------------------------------------
# 3️⃣ Extract 2D Footprint Polygon From Space
# --------------------------------------------------
def get_product_polygon(product, settings, simplify_tolerance=0.001):
    """
    Extracts the exact 2D footprint of an IfcEntity.
    Natively preserves concavities and internal holes by unioning mesh faces.
    """
    try:
        # Generate 3D shape from IFC element
        shape = ifcopenshell.geom.create_shape(settings, product)
        verts = shape.geometry.verts  # Flat list: [x1, y1, z1, x2, y2, z2...]
        faces = shape.geometry.faces  # Flat list: [v1, v2, v3, v1, v2, v3...]
        
        # Group vertices into 2D (x, y) tuples directly, dropping Z
        points_2d = [(verts[i], verts[i+1]) for i in range(0, len(verts), 3)]

        if not points_2d:
            return None

        triangles = []
        
        # Reconstruct 2D triangles from the mesh faces
        for i in range(0, len(faces), 3):
            p1 = points_2d[faces[i]]
            p2 = points_2d[faces[i+1]]
            p3 = points_2d[faces[i+2]]
            
            tri = Polygon([p1, p2, p3])
            
            # Filter out purely vertical faces (walls) which have ~0 area in 2D
            if tri.is_valid and tri.area > 1e-6:
                triangles.append(tri)

        if not triangles:
            return None
            
        # Union merges all triangles. Shapely automatically 
        # detects the outer boundary and hollows out the interiors (holes).
        exact_poly = unary_union(triangles)
        
        # Clean up any self-intersections or bowtie polygons 
        # that sometimes occur when flattening complex 3D meshes.
        if not exact_poly.is_valid:
            exact_poly = exact_poly.buffer(0)
            
        # Simplify straight edges. Removing collinear vertices 
        # dramatically speeds up the .buffer() and .intersects() math later.
        # (Assuming model in meters; use ~1.0 if model is in millimeters).
        if simplify_tolerance > 0:
            exact_poly = exact_poly.simplify(simplify_tolerance, preserve_topology=True)
        
        return exact_poly

    except Exception as e:
        print(f"Failed to extract geometry for product {product.GlobalId}: {e}")
        return None


# --------------------------------------------------
# 4️⃣ Detect Adjacency Using Polygon Contact
# --------------------------------------------------


def polygons_adjacent(poly1, poly2, tolerance=0.2):
    """
    Checks if two polygons are adjacent.
    Uses direct area calculation to filter out corner touches, 
    which is exponentially faster than generating bounding boxes.
    """
    if poly1 is None or poly2 is None:
        return False

    # (Keep this!) If they are further apart than the tolerance, skip the heavy math entirely.
    if poly1.distance(poly2) > tolerance:
        return False

    buf1 = poly1.buffer(tolerance)
    buf2 = poly2.buffer(tolerance)

    # Calculate the exact overlapping polygon(s)
    overlap = buf1.intersection(buf2)

    if overlap.is_empty:
        return False

    # Simply check if the total overlap area exceeds your threshold.
    min_intersection_area = (tolerance*2) ** 2
    # This natively handles MultiPolygons (disconnected chunks) because Shapely 
    # sums the area of all parts automatically.
    return overlap.area >= min_intersection_area


# --------------------------------------------------
# 5️⃣ Compute Adjacency For One Storey
# --------------------------------------------------

def compute_space_adjacency(spaces, settings, tolerance=0.2):

    polygons = {}

    for space in spaces:
        polygons[space.GlobalId] = get_product_polygon(space, settings)

    adjacency = set()

    for s1, s2 in combinations(spaces, 2):
        poly1 = polygons[s1.GlobalId]
        poly2 = polygons[s2.GlobalId]

        if polygons_adjacent(poly1, poly2, tolerance):
            # Sort the GlobalIds so (A, B) and (B, A) become identical
            unique_pair = tuple(sorted([s1.GlobalId, s2.GlobalId]))
            adjacency.add(unique_pair)

    return adjacency

# --------------------------------------------------
# Compute Space-to-Wall Adjacency
# --------------------------------------------------
def compute_space_wall_adjacency(spaces, walls, settings, tolerance=0.2):
    """
    Compares a list of spaces against a list of walls to find adjacencies.
    """
    adjacency = set()
    
    # Pre-calculate polygons to avoid redundant processing
    space_polys = {s.GlobalId: get_product_polygon(s, settings) for s in spaces}
    wall_polys = {w.GlobalId: get_product_polygon(w, settings) for w in walls}

    # Compare every space against every wall
    for space in spaces:
        for wall in walls:
            poly_s = space_polys[space.GlobalId]
            poly_w = wall_polys[wall.GlobalId]

            if polygons_adjacent(poly_s, poly_w, tolerance):
                # Sort the GlobalIds so (Space, Wall) and (Wall, Space) are handled uniformly
                unique_pair = tuple(sorted([space.GlobalId, wall.GlobalId]))
                adjacency.add(unique_pair)

    return adjacency


# --------------------------------------------------
# 6️⃣ Full IFC Adjacency Extraction
# --------------------------------------------------

def compute_ifc_adjacency(ifc_path, tolerance=0.2):

    ifc = ifcopenshell.open(ifc_path)
    settings = create_geometry_settings()

    storey_spaces = get_spaces_by_storey(ifc)
    storey_walls = get_walls_by_storey(ifc)

    # Use a master set to prevent cross-storey duplication
    all_adjacencies = set()

    for storey_id, spaces in storey_spaces.items():
        # Get walls for this specific storey (default to empty list if none exist)
        walls = storey_walls.get(storey_id, [])
        
        # 1. Calculate Space-to-Space Adjacencies
        if len(spaces) >= 2:
            storey_space_adj = compute_space_adjacency(spaces, settings, tolerance)
            all_adjacencies.update(storey_space_adj)

        # 2. Calculate Space-to-Wall Adjacencies
        if len(spaces) > 0 and len(walls) > 0:
            storey_space_wall_adj = compute_space_wall_adjacency(spaces, walls, settings, tolerance)
            all_adjacencies.update(storey_space_wall_adj)

    # Convert the final clean set back to a list for output
    return list(all_adjacencies)

In [15]:
def link_spaces(arc_ttl, adjacencies):
    """
    For paired spaces, link them using BOT.adjacentZone.
    """
    g = Graph()
    for p, ns in PREFIXES.items(): g.bind(p, ns)
    graph = Graph()    
    graph.parse(arc_ttl, format="ttl")    
    linked_count = 0  
    for (space_guid_0, space_guid_1) in adjacencies:
        space_uri_0 = next(
            graph.subjects(PROPS.globalIdIfcRoot_attribute_simple, Literal(space_guid_0)),
            None
        )
        if space_uri_0 is None:
            print(f"Warning: No URI found for space with GUID {space_guid_0}")
            continue

        space_uri_1 = next(
            graph.subjects(PROPS.globalIdIfcRoot_attribute_simple, Literal(space_guid_1)),
            None
        )
        if space_uri_1 is None:
            print(f"Warning: No URI found for space with GUID {space_guid_1}")
            continue

        g.add((space_uri_0, BOT.adjacentZone, space_uri_1))
        linked_count += 1
        
    print(f"Linked {linked_count} adjacent spaces in the graph.")
    return g

In [16]:
def link_spaces_walls(arc_ttl, adjacencies):
    """
    For paired spaces, link them using BOT.adjacentZone.
    🟢 MODIFIED: Added logic to also link spaces to walls using BOT.adjacentElement.
    """
    g = Graph()
    for p, ns in PREFIXES.items(): 
        g.bind(p, ns)
        
    graph = Graph()    
    graph.parse(arc_ttl, format="ttl")    
    
    linked_spaces_count = 0  
    linked_walls_count = 0 # 🟢 NEW: Track space-to-wall links

    for (guid_0, guid_1) in adjacencies:
        # 1. Fetch URI for the first GUID
        uri_0 = next(
            graph.subjects(PROPS.globalIdIfcRoot_attribute_simple, Literal(guid_0)),
            None
        )
        if uri_0 is None:
            print(f"Warning: No URI found for GUID {guid_0}")
            continue

        # 2. Fetch URI for the second GUID
        uri_1 = next(
            graph.subjects(PROPS.globalIdIfcRoot_attribute_simple, Literal(guid_1)),
            None
        )
        if uri_1 is None:
            print(f"Warning: No URI found for GUID {guid_1}")
            continue

        # 🟢 NEW: Determine the type of each URI to route to the correct BOT property
        type_0_uri = next(graph.objects(uri_0, RDF.type), None)
        type_1_uri = next(graph.objects(uri_1, RDF.type), None)
        
        # Safely convert to lowercase string to check if "space" is in the class name (e.g., IfcSpace)
        is_space_0 = type_0_uri and "space" in str(type_0_uri).lower()
        is_space_1 = type_1_uri and "space" in str(type_1_uri).lower()

        # 🟢 MODIFIED: Apply appropriate BOT relation based on types
        if is_space_0 and is_space_1:
            # Both are spaces
            g.add((uri_0, BOT.adjacentZone, uri_1))
            linked_spaces_count += 1
            
        elif is_space_0 and not is_space_1:
            # uri_0 is the Space, uri_1 is the Wall
            # Note: bot:adjacentElement strictly goes from Zone to Element
            g.add((uri_0, BOT.adjacentElement, uri_1))
            linked_walls_count += 1
            
        elif not is_space_0 and is_space_1:
            # uri_1 is the Space, uri_0 is the Wall
            g.add((uri_1, BOT.adjacentElement, uri_0))
            linked_walls_count += 1
            
        else:
            # (Optional) Both are walls; ignore or handle if needed
            pass

    print(f"Linked {linked_spaces_count} adjacent spaces (bot:adjacentZone).")
    print(f"Linked {linked_walls_count} space-to-wall boundaries (bot:adjacentElement).") # 🟢 NEW
    
    return g

In [17]:
def save_graph_to_file(graph, output_filename):
    graph.serialize(destination=output_filename, format="turtle")
    print(f"Final graph saved to {output_filename}")

In [18]:
arc_ifc_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\IFC\01ARK\ARK_MET.ifc"
arc_ttl_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\01ARK\ARK_MET.ttl"

In [34]:
adjacencies = compute_ifc_adjacency(arc_ifc_path, tolerance=0.2)
from pprint import pprint
print(f"Total number of adjacent pairs: {len(adjacencies)}")
pprint(adjacencies)

Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4MQ: Failed to process shape. Product: #375631=IfcSpace('3oZ3NR9tT0cvK6l5C9N4MQ',#41,'A2033','784',$,#375517,#375629,'Jätehuone',.ELEMENT.,.INTERNAL.,$), representation: #375627=IfcShapeRepresentation(#102,'Body','SweptSolid',(#375626))
Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4MQ: Failed to process shape. Product: #375631=IfcSpace('3oZ3NR9tT0cvK6l5C9N4MQ',#41,'A2033','784',$,#375517,#375629,'Jätehuone',.ELEMENT.,.INTERNAL.,$), representation: #375627=IfcShapeRepresentation(#102,'Body','SweptSolid',(#375626))
Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4Nf: Failed to process shape. Product: #375990=IfcSpace('3oZ3NR9tT0cvK6l5C9N4Nf',#41,'A3034','784',$,#375857,#375988,'Jätehuone',.ELEMENT.,.INTERNAL.,$), representation: #375986=IfcShapeRepresentation(#102,'Body','SweptSolid',(#375985))
Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4Nf: Failed to process shape. Product: #375990=IfcSpace('

In [35]:
space_wall_links = link_spaces_walls(arc_ttl_path, adjacencies)


Linked 2159 adjacent spaces (bot:adjacentZone).
Linked 11292 space-to-wall boundaries (bot:adjacentElement).


In [36]:
output_ttl_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\Linkset\linked_spaces_walls.ttl"
save_graph_to_file(space_wall_links, output_ttl_path)

Final graph saved to C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\MD2MV\data\TTL\Linkset\linked_spaces_walls.ttl


In [22]:
arc_ifc = ifcopenshell.open(arc_ifc_path)

In [32]:
storey_walls = get_product_by_storey(arc_ifc, "IfcWallStandardCase")
print(f"Total number of storeys with walls: {len(storey_walls)}")
pprint(storey_walls)

Total number of storeys with walls: 12
{'0eoqdncsb8SAoZq$ETCDRy': [],
 '0eoqdncsb8SAoZq$ETSjD$': [],
 '0eoqdncsb8SAoZq$ETSjDh': [],
 '0eoqdncsb8SAoZq$ETSjGN': [],
 '0eoqdncsb8SAoZq$ETSjGa': [],
 '0eoqdncsb8SAoZq$ETSjH9': [],
 '0eoqdncsb8SAoZq$ETSjI4': [],
 '0eoqdncsb8SAoZq$ETSjJ0': [],
 '0eoqdncsb8SAoZq$ETTO6Y': [],
 '0eoqdncsb8SAoZq$ETTO8Z': [],
 '0eoqdncsb8SAoZq$ETTygP': [],
 '0eoqdncsb8SAoZq$ETVUGE': []}


In [25]:
walls = arc_ifc.by_type("IfcWall")
print(f"Total number of walls in the model: {len(walls)}")

Total number of walls in the model: 7458


In [24]:
w2 = arc_ifc.by_type("IfcWallStandardCase")
print(f"Total number of IfcWallStandardCase in the model: {len(w2)}")

Total number of IfcWallStandardCase in the model: 7083


In [175]:
storeys = arc_ifc.by_type("IfcBuildingStorey")
print(f"Total storeys found: {len(storeys)}")   
pprint(storeys)

Total storeys found: 12
[#127=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTygP',#41,'Merenpinta',$,$,#125,$,'Merenpinta',.ELEMENT.,0.),
 #133=IfcBuildingStorey('0eoqdncsb8SAoZq$ETVUGE',#41,'R.kerros',$,$,#132,$,'R.kerros',.ELEMENT.,15899.9999999986),
 #139=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjH9',#41,'K.kerros',$,$,#138,$,'K.kerros',.ELEMENT.,19899.9999999986),
 #145=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjGa',#41,'0.kerros',$,$,#144,$,'0.kerros',.ELEMENT.,23700.),
 #151=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTO6Y',#41,'1.kerros',$,$,#150,$,'1.kerros',.ELEMENT.,27800.),
 #157=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTO8Z',#41,'2.kerros',$,$,#156,$,'2.kerros',.ELEMENT.,32700.0000000061),
 #163=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjD$',#41,'3.kerros',$,$,#162,$,'3.kerros',.ELEMENT.,36900.0000000061),
 #169=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjDh',#41,'4.kerros',$,$,#168,$,'4.kerros',.ELEMENT.,41100.0000000061),
 #175=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjI4',#41,'5.kerros',$,$,#174,$,'5.kerros',

In [176]:
st = storeys[0]
st.get_info()

{'id': 127,
 'type': 'IfcBuildingStorey',
 'GlobalId': '0eoqdncsb8SAoZq$ETTygP',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'Merenpinta',
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': #125=IfcLocalPlacement(#32,#124),
 'Representation': None,
 'LongName': 'Merenpinta',
 'CompositionType': 'ELEMENT',
 'Elevation': 0.0}

In [178]:
for i in range(len(storeys)-1):
    elevation_diff = storeys[i+1].Elevation - storeys[i].Elevation
    print(f"Storey: {storeys[i+1].Name} (Elevation: {storeys[i+1].Elevation}) - Previous: {storeys[i].Name} (Elevation: {storeys[i].Elevation}) = Difference: {elevation_diff}")

Storey: R.kerros (Elevation: 15899.9999999986) - Previous: Merenpinta (Elevation: 0.0) = Difference: 15899.9999999986
Storey: K.kerros (Elevation: 19899.9999999986) - Previous: R.kerros (Elevation: 15899.9999999986) = Difference: 4000.0
Storey: 0.kerros (Elevation: 23700.0) - Previous: K.kerros (Elevation: 19899.9999999986) = Difference: 3800.0000000014006
Storey: 1.kerros (Elevation: 27800.0) - Previous: 0.kerros (Elevation: 23700.0) = Difference: 4100.0
Storey: 2.kerros (Elevation: 32700.0000000061) - Previous: 1.kerros (Elevation: 27800.0) = Difference: 4900.000000006101
Storey: 3.kerros (Elevation: 36900.0000000061) - Previous: 2.kerros (Elevation: 32700.0000000061) = Difference: 4199.999999999996
Storey: 4.kerros (Elevation: 41100.0000000061) - Previous: 3.kerros (Elevation: 36900.0000000061) = Difference: 4200.0
Storey: 5.kerros (Elevation: 45300.0000000061) - Previous: 4.kerros (Elevation: 41100.0000000061) = Difference: 4200.0
Storey: 6.kerros (Elevation: 49500.0000000061) - Pr

In [179]:
stairs = arc_ifc.by_type("IfcStair")
print(f"Total stairs found: {len(stairs)}")
    

Total stairs found: 101


In [184]:
stair = stairs[0]
arc_ifc.traverse(stair)

[#1662794=IfcStair('3oZ3NR9tT0cvK6l5C9N4B0',#41,'Assembled Stair:Stair:666736',$,'Assembled Stair:sisäkierreporras:6978646',#1662793,$,'666736',.NOTDEFINED.),
 #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 #38=IfcPersonAndOrganization(#35,#37,$),
 #35=IfcPerson($,'','mariak',$,$,$,$,$),
 #37=IfcOrganization($,'','',$,$),
 #5=IfcApplication(#1,'2016','Autodesk Revit 2016 (ENU)','Revit'),
 #1=IfcOrganization($,'Autodesk Revit 2016 (ENU)',$,$,$),
 #1662793=IfcLocalPlacement(#138,#1662792),
 #138=IfcLocalPlacement(#32,#137),
 #32=IfcLocalPlacement(#23678880,#31),
 #23678880=IfcLocalPlacement($,#23678879),
 #23678879=IfcAxis2Placement3D(#23678877,$,$),
 #23678877=IfcCartesianPoint((-2.27093696594238E-06,-4.54187393188477E-06,0.)),
 #31=IfcAxis2Placement3D(#6,$,$),
 #6=IfcCartesianPoint((0.,0.,0.)),
 #137=IfcAxis2Placement3D(#135,$,$),
 #135=IfcCartesianPoint((0.,0.,19900.)),
 #1662792=IfcAxis2Placement3D(#6,$,$)]

In [183]:
xxx = arc_ifc.by_guid('3DlOKkwIf6vg32dUGVAICq')
xxx.get_info()

{'id': 23679198,
 'type': 'IfcRelContainedInSpatialStructure',
 'GlobalId': '3DlOKkwIf6vg32dUGVAICq',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': None,
 'Description': None,
 'RelatedElements': (#1482005=IfcWallStandardCase('2L_Z6exkTEph3i0Ee8PGV8',#41,'Basic Wall:VS VSS - 400 mm:481414',$,'Basic Wall:VS VSS - 400 mm:535067',#1481972,#1482003,'481414'),
  #1489053=IfcWallStandardCase('1j69kFL3DByvMx1erQyo25',#41,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):483062',$,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):1617896',#1489013,#1489051,'483062'),
  #1492846=IfcWallStandardCase('1j69kFL3DByvMx3enQyo2E',#41,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):483069',$,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):1617896',#1492826,#1492844,'483069'),
  #1492918=IfcWallStandardCase('1j69kFL3DByvMx1erQyo2D',#41,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):483070',$,'Basic Wall:VS6 (Kantava väliseinä, 250 mm):1617896',#1492894,#1492916,'483070'),
  #1493130=IfcWallStanda

In [29]:
room_pairs = []
for ad in adjacencies:
    s_ad0 = arc_ifc.by_guid(ad[0])
    s_ad1 = arc_ifc.by_guid(ad[1])
    room_pairs.append((s_ad0.Name, s_ad1.Name))
    print(f"Adjacent pair: {s_ad0.Name} <--> {s_ad1.Name}")

print(f"Total adjacent spaces: {len(room_pairs)}")

Adjacent pair: D4003 <--> D4002
Adjacent pair: B2501 <--> B2603
Adjacent pair: A1002 <--> A1003
Adjacent pair: B3029 <--> B3031
Adjacent pair: D2009 <--> D2502
Adjacent pair: A0501 <--> A0701
Adjacent pair: B1013.4 <--> B1013.1
Adjacent pair: B0503 <--> B0016
Adjacent pair: C0503 <--> D0001
Adjacent pair: B2702 <--> B2001
Adjacent pair: B3503 <--> B3501
Adjacent pair: B6503 <--> B6006
Adjacent pair: AK704 <--> AK707
Adjacent pair: BK016 <--> BK017
Adjacent pair: B0703 <--> B0041
Adjacent pair: B4503 <--> B4609
Adjacent pair: A5002 <--> A5501
Adjacent pair: AK003 <--> AK011
Adjacent pair: AK703 <--> AK704
Adjacent pair: B2501 <--> B2024
Adjacent pair: A0501 <--> A0702
Adjacent pair: BK701 <--> BK045
Adjacent pair: A0705 <--> A0703
Adjacent pair: A0706 <--> A0703
Adjacent pair: A1008 <--> A1003
Adjacent pair: D1008 <--> D1009
Adjacent pair: BK022 <--> BK024
Adjacent pair: B3701 <--> B3501
Adjacent pair: B6009 <--> B6018
Adjacent pair: C5501 <--> C5001
Adjacent pair: BK018 <--> BK016
Adja

In [167]:
def find_duplicates(pairs):
    seen = set()
    duplicates = set()

    for a, b in pairs:
        edge = frozenset((a, b))  # order-independent

        if edge in seen:
            duplicates.add(tuple(edge))
        else:
            seen.add(edge)

    return duplicates


In [168]:
repeated_pairs = find_duplicates(room_pairs)
print(f"Total repeated pairs: {len(repeated_pairs)}")
pprint(repeated_pairs)

Total repeated pairs: 0
set()


In [158]:
spaces = arc_ifc.by_type("IfcSpace")
for se in spaces:
    if se.Name == 'A1502':
        print(f"Found space: {se.Name} with GlobalId: {se.GlobalId}")

Found space: A1502 with GlobalId: 3VTYFK8vD6zQrOpleLa9Rd
Found space: A1502 with GlobalId: 2eKm1MRbj9lQ$fk6CjSQZn


In [154]:
print(list(uniqe_pairs)[0:5])

[('B6501', 'B6502'), ('A2014', 'A2015'), ('B6009', 'B6010'), ('A2001', 'A2002'), ('D3006', 'D3016')]


In [151]:
print(f"Total repeated pairs: {len(repeated_pairs)}")
pprint(repeated_pairs)

Total repeated pairs: 2375
{('A0001', 'A0502'),
 ('A0001', 'A0701'),
 ('A0002', 'A0006'),
 ('A0002', 'A0007'),
 ('A0002', 'A0008'),
 ('A0002', 'A0502'),
 ('A0003', 'A0503'),
 ('A0003', 'A0708'),
 ('A0004', 'A0707'),
 ('A0005', 'A0503'),
 ('A0005', 'A0708'),
 ('A0006', 'A0002'),
 ('A0007', 'A0002'),
 ('A0008', 'A0002'),
 ('A0009', 'A0504'),
 ('A0010', 'A0504'),
 ('A0011', 'A0012'),
 ('A0011', 'A0503'),
 ('A0011', 'A0506'),
 ('A0011', 'A0608'),
 ('A0011', 'B0506'),
 ('A0012', 'A0011'),
 ('A0502', 'A0001'),
 ('A0502', 'A0002'),
 ('A0502', 'A0503'),
 ('A0502', 'A0708'),
 ('A0503', 'A0003'),
 ('A0503', 'A0005'),
 ('A0503', 'A0011'),
 ('A0503', 'A0502'),
 ('A0504', 'A0009'),
 ('A0504', 'A0010'),
 ('A0504', 'A0709'),
 ('A0504', 'A0710'),
 ('A0504', 'B0505'),
 ('A0505', 'A0602'),
 ('A0505', 'A0608'),
 ('A0506', 'A0011'),
 ('A0506', 'B0706'),
 ('A0602', 'A0505'),
 ('A0604', 'A0605'),
 ('A0605', 'A0604'),
 ('A0608', 'A0011'),
 ('A0608', 'A0505'),
 ('A0701', 'A0001'),
 ('A0703', 'A0705'),
 ('A070

In [26]:
import ifcopenshell
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon

# --------------------------------------------------
# 🎨 Visualize Two Spaces by GUID (Updated for Holes)
# --------------------------------------------------
def visualize_space_relation(ifc, guid1, guid2, tolerance=5.0):
    # 1. Load the IFC model and geometry settings
    
    settings = create_geometry_settings()

    # 2. Fetch the spaces using their GUIDs
    try:
        space1 = ifc.by_guid(guid1)
        space2 = ifc.by_guid(guid2)
    except Exception as e:
        print(f"Error finding GUIDs: {e}")
        return

    # 3. Extract exact 2D polygons (includes holes and concavities)
    poly1 = get_product_polygon(space1, settings)
    poly2 = get_product_polygon(space2, settings)

    if poly1 is None or poly2 is None:
        print("Could not extract geometry for one or both spaces.")
        return

    # 4. Calculate the buffered geometry and intersections
    buf1 = poly1.buffer(tolerance)
    buf2 = poly2.buffer(tolerance)
    
    is_adjacent = buf1.intersects(buf2)
    intersection = buf1.intersection(buf2)

    # 5. Set up Matplotlib plot
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # 🟢 MODIFIED: Upgraded helper function to handle MultiPolygons and Holes
    def plot_poly(ax, poly, edge_color, fill_color, alpha, linestyle='-'):
        if poly is None or poly.is_empty:
            return
            
        # Normalize to a list to easily handle both Polygon and MultiPolygon
        geoms = poly.geoms if poly.geom_type in ('MultiPolygon', 'GeometryCollection') else [poly]
        
        for geom in geoms:
            # 1. Plot the exterior boundary
            x, y = geom.exterior.xy
            ax.plot(x, y, color=edge_color, linestyle=linestyle, linewidth=2)
            ax.fill(x, y, color=fill_color, alpha=alpha)
            
            # 2. 🟢 MODIFIED: Plot the interior boundaries (holes)
            for interior in geom.interiors:
                xi, yi = interior.xy
                # Draw the hole's outline
                ax.plot(xi, yi, color=edge_color, linestyle=linestyle, linewidth=1.5)
                # Fill the hole with white to "punch it out" visually against the colored space
                ax.fill(xi, yi, color='white', alpha=1.0)

    # Plot original exact polygons
    plot_poly(ax, poly1, edge_color='blue', fill_color='dodgerblue', alpha=0.4)
    plot_poly(ax, poly2, edge_color='green', fill_color='limegreen', alpha=0.4)

    # Plot buffered polygons (dotted lines to show the tolerance reach)
    plot_poly(ax, buf1, edge_color='blue', fill_color='none', alpha=0.5, linestyle=':')
    plot_poly(ax, buf2, edge_color='green', fill_color='none', alpha=0.5, linestyle=':')

    # Highlight intersection if they are adjacent
    if is_adjacent and not intersection.is_empty:
        plot_poly(ax, intersection, edge_color='red', fill_color='red', alpha=0.6)
        title_status = "ADJACENT"
    else:
        title_status = "NOT ADJACENT"

    # Formatting the plot
    ax.set_title(f"Space Relation: {guid1} vs {guid2}\nStatus: {title_status} (Tolerance: {tolerance})")
    ax.set_aspect('equal') # Ensures the geometry isn't stretched
    ax.set_xlabel("X coordinate")
    ax.set_ylabel("Y coordinate")
    
    # Push the grid to the back so it doesn't overlap weirdly with the white hole fills
    ax.set_axisbelow(True) 
    ax.grid(True, linestyle='--', alpha=0.6)
    
    # Show the plot window
    plt.show()

In [159]:
r1 = arc_ifc.by_guid('3VTYFK8vD6zQrOpleLa9Rd')
r2 = arc_ifc.by_guid('2eKm1MRbj9lQ$fk6CjSQZn')

In [160]:
r1.get_info()

{'id': 104011,
 'type': 'IfcSpace',
 'GlobalId': '3VTYFK8vD6zQrOpleLa9Rd',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A1502',
 'Description': '899',
 'ObjectType': None,
 'ObjectPlacement': #85729=IfcLocalPlacement(#150,#85728),
 'Representation': #104009=IfcProductDefinitionShape($,$,(#104007)),
 'LongName': 'Ruokailutilat',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [161]:
r2.get_info()

{'id': 582749,
 'type': 'IfcSpace',
 'GlobalId': '2eKm1MRbj9lQ$fk6CjSQZn',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A1502',
 'Description': '899',
 'ObjectType': None,
 'ObjectPlacement': #564415=IfcLocalPlacement(#150,#564414),
 'Representation': #582747=IfcProductDefinitionShape($,$,(#582745)),
 'LongName': 'Ruokailutilat',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [37]:
(g1, g2) = ('3oZ3NR9tT0cvK6l5C9N4MQ','3oZ3NR9tT0cvK6l5C9N4Nf')
#g2 = '338qDR18z3PBg6oqz1i4QY'
r1 = arc_ifc.by_guid(g1)
print(r1.Name)
r2 = arc_ifc.by_guid(g2)
print(r2.Name)

A2033
A3034


In [38]:
visualize_space_relation(arc_ifc, g1, g2, tolerance=0.2)

Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4MQ: Failed to process shape. Product: #375631=IfcSpace('3oZ3NR9tT0cvK6l5C9N4MQ',#41,'A2033','784',$,#375517,#375629,'Jätehuone',.ELEMENT.,.INTERNAL.,$), representation: #375627=IfcShapeRepresentation(#102,'Body','SweptSolid',(#375626))
Failed to extract geometry for product 3oZ3NR9tT0cvK6l5C9N4Nf: Failed to process shape. Product: #375990=IfcSpace('3oZ3NR9tT0cvK6l5C9N4Nf',#41,'A3034','784',$,#375857,#375988,'Jätehuone',.ELEMENT.,.INTERNAL.,$), representation: #375986=IfcShapeRepresentation(#102,'Body','SweptSolid',(#375985))
Could not extract geometry for one or both spaces.


In [61]:
r1.get_info()

{'id': 582749,
 'type': 'IfcSpace',
 'GlobalId': '2eKm1MRbj9lQ$fk6CjSQZn',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A1502',
 'Description': '899',
 'ObjectType': None,
 'ObjectPlacement': #564415=IfcLocalPlacement(#150,#564414),
 'Representation': #582747=IfcProductDefinitionShape($,$,(#582745)),
 'LongName': 'Ruokailutilat',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [62]:
r2.get_info()

{'id': 104011,
 'type': 'IfcSpace',
 'GlobalId': '3VTYFK8vD6zQrOpleLa9Rd',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A1502',
 'Description': '899',
 'ObjectType': None,
 'ObjectPlacement': #85729=IfcLocalPlacement(#150,#85728),
 'Representation': #104009=IfcProductDefinitionShape($,$,(#104007)),
 'LongName': 'Ruokailutilat',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [45]:
space_storey = get_spaces_by_storey(arc_ifc)
print(f"Total number of storeys: {len(space_storey)}")
for ss in space_storey:
    print(f"Storey {ss} has {len(space_storey[ss])} spaces.")

Total number of storeys: 12
Storey 0eoqdncsb8SAoZq$ETTygP has 0 spaces.
Storey 0eoqdncsb8SAoZq$ETVUGE has 0 spaces.
Storey 0eoqdncsb8SAoZq$ETSjH9 has 119 spaces.
Storey 0eoqdncsb8SAoZq$ETSjGa has 119 spaces.
Storey 0eoqdncsb8SAoZq$ETTO6Y has 199 spaces.
Storey 0eoqdncsb8SAoZq$ETTO8Z has 165 spaces.
Storey 0eoqdncsb8SAoZq$ETSjD$ has 183 spaces.
Storey 0eoqdncsb8SAoZq$ETSjDh has 171 spaces.
Storey 0eoqdncsb8SAoZq$ETSjI4 has 174 spaces.
Storey 0eoqdncsb8SAoZq$ETSjJ0 has 102 spaces.
Storey 0eoqdncsb8SAoZq$ETSjGN has 4 spaces.
Storey 0eoqdncsb8SAoZq$ETCDRy has 0 spaces.


In [5]:
spaces = arc_ifc.by_type("IfcSpace")
for space in spaces[:5]:  # Process only the first 5 spaces for demonstration
    print(f"Space Name: {space.Name}, GlobalId: {space.GlobalId}")

Space Name: A2030, GlobalId: 1X7gk6z8b6NhpPEZsx$9cM
Space Name: BK003, GlobalId: 3v2__NkbfF5gKILhSKNZEC
Space Name: BK008, GlobalId: 3v2__NkbfF5gKILhSKNZCd
Space Name: BK031, GlobalId: 3v2__NkbfF5gKILhSKNZCf
Space Name: BK029, GlobalId: 3v2__NkbfF5gKILhSKNZCq


In [32]:
storeys = arc_ifc.by_type("IfcBuildingStorey")
storey = storeys[0]
arc_ifc.get_inverse(storey)

{#23697062=IfcRelAggregates('27PCKGLxT4mxtV9cw6mgBW',#41,$,$,#118,(#127,#133,#139,#145,#151,#157,#163,#169,#175,#181,#187,#193)),
 #23678968=IfcRelContainedInSpatialStructure('3HoIA6Wln0EActgji30evZ',#41,$,$,(#1938222,#5146963,#6458493,#6458526,#8752623,#10747967,#12302363,#12302473,#12302529,#12302976,#12303314,#12303542,#12304097,#12304466,#12305917,#12305971,#12306436,#12307142,#12307598,#12308062,#12308613,#12308692,#12308773,#12308849,#12310731,#12310914,#12311227,#12311494,#12311802,#12312288,#12312560,#12312653,#12312756,#12313029,#12313082,#12313263,#12342364,#12342481,#12342663,#12342810,#12342884,#12711281,#12711312,#12860122,#12865575,#12868198,#12872812,#13266772,#13273883,#13274175,#13274205,#13551494,#13556240,#13636111,#13636131,#13636152,#13636173,#13636193,#13636214,#13636234,#13636255,#13636286,#13636307,#13636327,#13636348,#13636369,#13917200,#13917220,#14085160,#14112046,#14205591,#14217416,#14217436,#14217456,#14217477,#14217498,#14217519,#14219592,#15078368,#15201

In [6]:
s = spaces[0]
s.get_info()

{'id': 1029,
 'type': 'IfcSpace',
 'GlobalId': '1X7gk6z8b6NhpPEZsx$9cM',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': 'A2030',
 'Description': '',
 'ObjectType': None,
 'ObjectPlacement': #196=IfcLocalPlacement(#156,#195),
 'Representation': #1025=IfcProductDefinitionShape($,$,(#1022)),
 'LongName': 'Luova ilmaisu, draama ja musiikki, 35 op',
 'CompositionType': 'ELEMENT',
 'InteriorOrExteriorSpace': 'INTERNAL',
 'ElevationWithFlooring': None}

In [7]:
arc_ifc.get_inverse(s)

{#23696134=IfcRelAggregates('15Z0v90RiHrPC200A6FoKR',#41,$,$,#157,(#1029,#61185,#61406,#61520,#67518,#70041,#70241,#72349,#73105,#73199,#73291,#73393,#73485,#73730,#73991,#74163,#74424,#74706,#79559,#110073,#111284,#126337,#126607,#126717,#127531,#127622,#127721,#127823,#127915,#128022,#128129,#128221,#135202,#135551,#136851,#137630,#137729,#138147,#142410,#142510,#142925,#143153,#143256,#143365,#143477,#144135,#144364,#144474,#144855,#147743,#148487,#161113,#161219,#161878,#162182,#162469,#163510,#163708,#166556,#168165,#173689,#197234,#197333,#197428,#316007,#316412,#320190,#322542,#324734,#327151,#329553,#330585,#335249,#344111,#344764,#348640,#349808,#372642,#375631,#380861,#381668,#386416,#392063,#392167,#392513,#392762,#401048,#402330,#418841,#422015,#422118,#422227,#422423,#422608,#422710,#422828,#423040,#423149,#429763,#432986,#433917,#434020,#434122,#434299,#434402,#434580,#437166,#444097,#444200,#444347,#444451,#444601,#444715,#444811,#444910,#445019,#445127,#445235,#445344,#

In [33]:
ele = arc_ifc.by_guid('27PCKGLxT4mxtV9cw6mgBW')
ele.get_info()

{'id': 23697062,
 'type': 'IfcRelAggregates',
 'GlobalId': '27PCKGLxT4mxtV9cw6mgBW',
 'OwnerHistory': #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 'Name': None,
 'Description': None,
 'RelatingObject': #118=IfcBuilding('0eoqdncsb8SAoZq$DYYdza',#41,'',$,$,#32,$,'',.ELEMENT.,$,$,#114),
 'RelatedObjects': (#127=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTygP',#41,'Merenpinta',$,$,#125,$,'Merenpinta',.ELEMENT.,0.),
  #133=IfcBuildingStorey('0eoqdncsb8SAoZq$ETVUGE',#41,'R.kerros',$,$,#132,$,'R.kerros',.ELEMENT.,15899.9999999986),
  #139=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjH9',#41,'K.kerros',$,$,#138,$,'K.kerros',.ELEMENT.,19899.9999999986),
  #145=IfcBuildingStorey('0eoqdncsb8SAoZq$ETSjGa',#41,'0.kerros',$,$,#144,$,'0.kerros',.ELEMENT.,23700.),
  #151=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTO6Y',#41,'1.kerros',$,$,#150,$,'1.kerros',.ELEMENT.,27800.),
  #157=IfcBuildingStorey('0eoqdncsb8SAoZq$ETTO8Z',#41,'2.kerros',$,$,#156,$,'2.kerros',.ELEMENT.,32700.0000000061),
  #163=IfcBuildingStorey(

In [18]:
print(len(ele.RelatedObjects))

798


In [15]:
arc_ifc.traverse(s, max_levels=2)

[#1029=IfcSpace('1X7gk6z8b6NhpPEZsx$9cM',#41,'A2030','',$,#196,#1025,'Luova ilmaisu, draama ja musiikki, 35 op',.ELEMENT.,.INTERNAL.,$),
 #41=IfcOwnerHistory(#38,#5,$,.NOCHANGE.,$,$,$,0),
 #38=IfcPersonAndOrganization(#35,#37,$),
 #5=IfcApplication(#1,'2016','Autodesk Revit 2016 (ENU)','Revit'),
 #196=IfcLocalPlacement(#156,#195),
 #156=IfcLocalPlacement(#32,#155),
 #195=IfcAxis2Placement3D(#6,$,$),
 #1025=IfcProductDefinitionShape($,$,(#1022)),
 #1022=IfcShapeRepresentation(#102,'Body','Brep',(#1021))]

In [13]:
ifcopenshell.util.element.get_psets(s)

{'Pset_SpaceCommon': {'Reference': 'Luova ilmaisu, draama ja musiikki, 35 op A2030',
  'Category': 'Rooms',
  'CeilingCovering': 'AK2 + AK11b',
  'WallCovering': 'SP01, SP02',
  'FloorCovering': 'LP01.1',
  'id': 1039},
 'Constraints': {'Base Offset': 0.0,
  'Level': '2.kerros',
  'Limit Offset': 3785.0,
  'Upper Limit': 'Level: 2.kerros',
  'id': 1105},
 'Dimensions': {'Area': 90.5110311340568,
  'Computation Height': 0.0,
  'Perimeter': 40527.146811265,
  'Unbounded Height': 3785.0,
  'Volume': 312.59627190209,
  'id': 1110},
 'Energy Analysis': {'Actual Lighting Load': 0.0,
  'Actual Lighting Load per area': 0.0,
  'Actual Power Load': 0.0,
  'Actual Power Load per area': 0.0,
  'Area per Person': 28.5714285714286,
  'Base Lighting Load on': '<Default>',
  'Base Power Load on': '<Default>',
  'Heat Load Values': '<Default>',
  'Latent Heat Gain per person': 630.91814901261,
  'Lighting Load Units': 'Power Density',
  'Number of People': 0.0,
  'Plenum Lighting Contribution': 0.2,
  